# Test PySyft x F2D

In [1]:
import syft as sy

In [2]:
sy.load("sklearn")
sy.load("pandas")
sy.load("numpy")

In [3]:
sy.load("fhir2dataset")

In [4]:
remote = sy.VirtualMachine().get_root_client()

[2021-04-07T15:04:47.519872+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Omniglot not supported in 0.7.0
[2021-04-07T15:04:47.520542+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Omniglot.__len__ not supported in 0.7.0
[2021-04-07T15:04:47.521001+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Places365 not supported in 0.7.0
[2021-04-07T15:04:47.521403+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Places365.__len__ not supported in 0.7.0
[2021-04-07T15:04:47.522988+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.GaussianBlur not supported in 0.7.0
[2021-04-07T15:04:47.523859+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.ConvertImageDtype not supported in 0.7.0
[2021-04-07T15:04:47.524605+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.functional.adjust_brightness not supported in 0.7.0
[2021

In [5]:
from fhir2dataset import Query


query = Query(fhir_api_url="http://hapi.fhir.org/baseR4/")

## Test serialization

In [6]:
d = sy.serialize(obj=query)

In [7]:
type(d)

proto.lib.fhir2dataset.query_pb2.FHIRQuery

In [8]:
sy.deserialize(blob=sy.serialize(obj=query))

# Create remotely a Query

In [10]:
sql_query = """
SELECT
Patient.birthDate,
Condition.code.coding.display
FROM Condition
INNER JOIN Patient 
ON Condition.subject = Patient._id
WHERE Patient.birthdate=ge2000-01-01
AND Patient.birthdate=le2000-04-09
"""

In [11]:
remote_query = remote.fhir2dataset.Query(fhir_api_url="http://hapi.fhir.org/baseR4/")
remote_df = remote_query.from_sql(sql_query).execute()

100%|█████████▉| 100.00/100.00


In [12]:
remote_counts = remote_df["Condition:Condition.code.coding.display"].value_counts()
remote_counts.get()

rheumatoid arthritis                       80
onychomycosis                              40
asthma                                     40
GERD                                       40
neuropathic pain                           40
diabetes                                   40
depression                                 40
hypertension                               40
high cholesterol                           40
angina                                     40
Normal pregnancy                            5
Pointed Ear                                 2
Streptococcal sore throat (disorder)        1
Fever (finding)                             1
Congenital pointed ear                      1
COVID-19                                    1
Viral sinusitis (disorder)                  1
Loss of taste (finding)                     1
Nasal congestion (finding)                  1
Sore throat symptom (finding)               1
Suspected COVID-19                          1
Hypertension                      

In [13]:
df = remote_df.get()
df

,Condition:from_id,Condition:Condition.code.coding.display,Patient:from_id,Patient:Patient.birthDate
0,Condition/1660458,GERD,Patient/1660449,2000-01-01
1,Condition/1660457,rheumatoid arthritis,Patient/1660449,2000-01-01
2,Condition/1660459,high cholesterol,Patient/1660449,2000-01-01
3,Condition/1660461,angina,Patient/1660449,2000-01-01
4,Condition/1660460,onychomycosis,Patient/1660449,2000-01-01
...,...,...,...,...
456,Condition/1499252,rheumatoid arthritis,Patient/1499244,2000-02-01
457,Condition/1499251,rheumatoid arthritis,Patient/1499244,2000-02-01
458,Condition/1499254,high cholesterol,Patient/1499244,2000-02-01
459,Condition/1499253,GERD,Patient/1499244,2000-02-01


In [14]:
type(df["Condition:Condition.code.coding.display"].value_counts())

pandas.core.series.Series

# Send a query

In [15]:
alice_vm = sy.VirtualMachine(name="alice")

# Get clients from each VM
alice = alice_vm.get_root_client()

query = Query(fhir_api_url="http://hapi.fhir.org/baseR4/")
query.tag('#f2d')

query_ptr = query.send(alice)

[2021-04-07T15:05:01.713232+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Omniglot not supported in 0.7.0
[2021-04-07T15:05:01.713835+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Omniglot.__len__ not supported in 0.7.0
[2021-04-07T15:05:01.714313+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Places365 not supported in 0.7.0
[2021-04-07T15:05:01.714701+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.datasets.Places365.__len__ not supported in 0.7.0
[2021-04-07T15:05:01.716242+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.GaussianBlur not supported in 0.7.0
[2021-04-07T15:05:01.716922+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.ConvertImageDtype not supported in 0.7.0
[2021-04-07T15:05:01.717359+0200][CRITICAL][logger]][98023] Skipping torchvision.torchvision.transforms.functional.adjust_brightness not supported in 0.7.0
[2021

In [16]:
query_ptr

In [17]:
alice.store[0].tags

['#f2d']

In [18]:
query_back = query_ptr.get()

In [19]:
query_back

In [20]:
query_back.tags

['#f2d']